Код содержит алгоритмы для рекомендации пользователям музыкальных композиций на основе общего рейтинга популярности песен и индивидуальных предпочтений позователей, определеяемых на основе предшествующей истории прослушиваний.
Алгоритмы реализованы в формате класса Recommender() с несколькими методами.

In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
# Файл с данными по ID пользователя, ID песни и количеству прослушиваний:
listening_data = 'https://static.turi.com/datasets/millionsong/10000.txt'
song_df = pd.read_table(listening_data, header=None, names = ['user_id', 'song_id', 'listenings'])
song_df.shape

(2000000, 3)

In [3]:
# Верх 1-й таблицы:
song_df.head()

,user_id,song_id,listenings
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1


In [4]:
# Файл с данными о песнях и исполнителях:
song_data = 'https://static.turi.com/datasets/millionsong/song_data.csv'
song_df_2 =  pd.read_csv(song_data)
song_df_2.shape

(1000000, 5)

In [5]:
# Верх 2-й таблицы:
song_df_2.head()

,song_id,title,release,artist_name,year
0,SOQMMHC12AB0180CB8,Silent Night,Monster Ballads X-Mas,Faster Pussy cat,2003
1,SOVFVAK12A8C1350D9,Tanssi vaan,Karkuteillä,Karkkiautomaatti,1995
2,SOGTUKN12AB017F4F1,No One Could Ever,Butter,Hudson Mohawke,2006
3,SOBNYVR12A8C13558C,Si Vos Querés,De Culo,Yerba Brava,2003
4,SOHSBXH12A8C13B0DF,Tangle Of Aspens,Rene Ablaze Presents Winter Sessions,Der Mystic,0


In [6]:
# Объединяем данные в общую базу по ID песни (некоторые строки имеют повторяющиеся 'song_id'):
song_df = pd.merge(song_df, song_df_2.drop_duplicates(['song_id']), on='song_id', how='left')
song_df.shape

(2000000, 7)

In [7]:
# Устраняем техническую ошибку в базе (заменяем значения 0 в столбце 'year' на np.nan):
song_df.loc[song_df['year'] == 0, 'year'] = np.nan

In [8]:
# Верх объединенной таблицы:
song_df.head()

,user_id,song_id,listenings,title,release,artist_name,year
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,The Cove,Thicker Than Water,Jack Johnson,NaN
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976.0
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,Stronger,Graduation,Kanye West,2007.0
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1,Constellations,In Between Dreams,Jack Johnson,2005.0
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1,Learn To Fly,There Is Nothing Left To Lose,Foo Fighters,1999.0


In [9]:
# Создаем базу песен, отсортированных по общему числу прослушиваний:
popular_df = song_df.groupby(['artist_name', 'title']).agg({'listenings': 'sum'}).reset_index()
popular_df.sort_values(by='listenings', ascending=False, inplace=True)
popular_df.reset_index(inplace=True, drop=True)
popular_df.head(10)

,artist_name,title,listenings
0,Dwight Yoakam,You're The One,54136
1,Björk,Undo,49253
2,Kings Of Leon,Revelry,41418
3,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Horn Concerto No. 4 in E flat K495: II. Romanc...,31153
4,Harmonia,Sehr kosmisch,31036
5,Florence + The Machine,Dog Days Are Over (Radio Edit),26663
6,OneRepublic,Secrets,22100
7,Five Iron Frenzy,Canada,21019
8,Tub Ring,Invalid,19645
9,Sam Cooke,Ain't Misbehavin,18309


In [10]:
# Список всех пользователей:
users = song_df['user_id'].unique()
print(len(users))

76353


In [11]:
# Список всех песен:
songs = song_df['song_id'].unique()
print(len(songs))

10000


In [12]:
class Recommender():
    """Класс позволяет формировать списки рекомендуемых пользователям песен
    на основе общего рейтинга популярности и предшествующей истории прослушиваний пользователя.
    После создания экземпляра класса необходимо вызвать функцию create_user(),
    передав ей ID текущего пользователя: это обновит атрибуты класса.
    Функцию можно вызывать многократно.
    Список рекомендуемых песен для текущего пользователя формируется
    через вызов одной из пяти функция: by_popularity(), by_singers(),
    by_release(), by_similar_users() и by_period()."""
    
    def __init__(self):
        """Инициализация экземпляра класса не требует передачи параметров.
        Атрибуты класса задаются при вызове функции create_user()
        и относятся к текущему пользователю. Методы класса производят
        операции фильтрации и сортировки на двух датафреймах,
        присвоенных ранее глобальным переменным 'song_df' и 'popular_df'."""
        self.user = None  # ID текущего пользователя
        self.user_data = None  # база прослушиваний пользователя
        self.singers = None  # исполнители, прослушанные пользователем
        self.songs = None  # песни, прослушанные пользователем
        self.time_start = None  # нижняя граница временного интервала прослушанных песен
        self.time_finish = None  # верхняя граница временного интервала прослушанных песен
        self.similar_users = None  # список пользователей с похожей историей прослушиваний
    
    def create_user(self, user_id):
        """Функция принимает ID пользователя и задает соответствующие ему атрибуты класса.
        Выводит на экран основные сведения о текущем пользователе: ID, прослушанные песни,
        исполнители, временной интервал, список похожих пользователей."""
        self.user = user_id
        # Фильтруем глобальную базу по ID текущего пользователя:
        self.user_data = song_df.loc[song_df['user_id'] == self.user, :].sort_values(by='listenings')
        
        # Составляем список исполнителей, которых прослушал пользователь:
        self.singers = self.user_data['artist_name'].unique()
        # Составляем список песен, которые прослушал пользователь:
        self.songs = self.user_data['title'].unique()
        
        # Самый ранний год выпуска среди песен, прослушанных пользователем:
        self.time_start = self.user_data['year'].min()
        # Самый поздний год выпуска среди песен, прослушанных пользователем:
        self.time_finish = self.user_data['year'].max()
        
        # Фильтруем глобальную базу по названиям песен из списка self.songs:
        song_df_filt = song_df.loc[song_df['title'].isin(self.songs), :].copy()
        # Убираем из этой базы текущего пользователя:
        song_df_filt = song_df_filt.loc[song_df_filt['user_id'] != self.user, :]
        # Создаем столбец для идентицикации совпадающих песен
        song_df_filt['matches'] = 0
        song_df_filt.loc[song_df_filt['title'].isin(self.songs), 'matches'] = 1
        # Группируем данные по ID пользователей с суммированием совпадений:
        self.similar_users = song_df_filt.groupby(['user_id']).agg({'matches': 'sum'}).reset_index()
        # Оставляем только пользователей, которые слушали не менее 20% репертуара текущего пользователя:
        similarity_check = len(self.songs) // 5
        self.similar_users = self.similar_users.loc[self.similar_users['matches'] >= similarity_check, 'user_id']
        
        # Выводим на экран основные сведения о текущем пользователе:
        print(f'Текущий пользователь: {self.user}')
        print(f'\nПрослушанные исполнители ({len(self.singers)}):\n{self.singers}')
        print(f'\nПрослушанные песни ({len(self.songs)} ед.):\n{self.songs}')
        print(f'\nВременной интервал: с {self.time_start} по {self.time_finish}')
        print(f'\nПохожие пользователи ({len(self.similar_users)} чел.):\n{self.similar_users}')
        
    def by_popularity(self, top_limit):
        """Функция принимает аргумент 'top_limit', определяющий диапазон рейтинга
        наиболее популярных песен, среди которых будут выбраны рекомендуемые.
        Функция выводит на экран список из 10 случайно подобранных песен,
        входящих в рейтинг наиболее популярных по числу прослушиваний."""
        # В базе 10 тыс. песен. Вводим проверку введенного аргумента на соответствие длине датафрейма:
        if top_limit <= len(popular_df):
            indexes = [random.randint(0, top_limit - 1) for _ in range(10)]  # 10 случайных чисел от 0 до 'top_limit'
        else:  # Если аргумент превышает длину датафрейма, выбираем 10 чисел в пределах существующих индексов:
            indexes = [random.randint(0, len(popular_df) - 1) for _ in range(10)]
        print(popular_df.loc[indexes, ['artist_name', 'title']])
        
    def by_singers(self):
        """Функция выводит на экран список из 10 песен, рекомендуемых пользователю
        на основе истории его прослушиваний. Песни выбираются из репертуара исполнителей,
        входящих в список 'self.singers' случайным образом и могут повторяться."""
        # Фильтруем базу песен по исполнителям, которых слушал пользователь:
        options = popular_df.loc[popular_df['artist_name'].isin(self.singers), ['artist_name', 'title']]
        # Выбираем 10 случайных строк из полученной таблицы:
        indexes = [random.randint(0, len(options) - 1) for _ in range(10)]
        print(options.iloc[indexes, :])
    
    def by_release(self):
        """Функция выводит на экран список из 10 песен, рекомендуемых пользователю
        на основе истории его прослушиваний. Выбираются композиции из релизов,
        песни из которых пользователь уже слушал. Список формируется случайным образом.
        Композиции могут повторяться, особенно если у пользователя ограниченная история прослушиваний."""
        # Фильтруем базу по песням, которые слушал пользователь, и составляем список релизов:
        releases = song_df.loc[song_df['title'].isin(self.songs), 'release'].unique()
        # Фильтруем базу песен по релизам:
        options = song_df.loc[song_df['release'].isin(releases), ['artist_name', 'title']]
        # Выбираем 10 случайных строк из полученной таблицы:
        indexes = [random.randint(0, len(options) - 1) for _ in range(10)]
        print(options.iloc[indexes, :])
    
    def by_similar_users(self):
        """Функция выводит на экран список из 10 песен, рекомендуемых пользователю
        на основе истории его прослушиваний. Песни выбираются из числа востребованных
        пользователями из списка self.similar_users. Список песен формируется случайным образом."""
        # Если атрибут 'self.similar_users' содержит значения:
        if len(self.similar_users) > 0:
            # Фильтруем глобальную базу по списку 'self.similar_users':
            options = song_df.loc[song_df['user_id'].isin(self.similar_users), ['artist_name', 'title']]
            # Выбираем 10 случайных строк из полученной таблицы:
            indexes = [random.randint(0, len(options) - 1) for _ in range(10)]
            print(options.iloc[indexes, :])
        else:
            print('Функция не может быть применена: нет пользователей со схожими предпочтениями.')
        
    def by_period(self):
        """Функция выводит на экран список из 10 песен, рекомендуемых пользователю
        на основе истории его прослушиваний. Случайным образом выбираются песни, 
        вышедшие в интервале между self.time_start и self.time_finish."""
        # Фильтруем глобальную базу по времени выхода песни:
        options = song_df.loc[(song_df['year'] >= self.time_start) & (song_df['year'] <= self.time_finish), 
                              ['artist_name', 'title']]
        # Выбираем 10 случайных строк из полученной таблицы:
        indexes = [random.randint(0, len(options) - 1) for _ in range(10)]
        print(options.iloc[indexes, :])

In [13]:
# Создаем экземпляр класса:
rec = Recommender()

In [14]:
# Передаем сведения об ID пользователя:
rec.create_user(users[0])  # первый пользователь списка

Текущий пользователь: b80344d063b5ccb3212f76538f3d9e43d87dca9e

Прослушанные исполнители (28):
['Jack Johnson' 'Angus & Julia Stone' 'Incubus' 'John Mayer'
 'Jimmy Eat World' 'Thievery Corporation' 'King Curtis' 'Band Of Horses'
 'Foo Fighters' 'Fleet Foxes' 'Kings Of Leon' 'The String Cheese Incident'
 'Sublime' 'Jack Johnson / Paula Fuga' 'Andrew Bird'
 'Jack Johnson / Matt Costa / Zach Gill / Dan Lebowitz / Steve Adams'
 'Kanye West' 'Lady GaGa' 'Harmonia'
 'Thievery Corporation feat. Emiliana Torrini' 'The Lonely Island'
 "The B-52's" 'Jorge Drexler' 'Paco De Lucia' 'Puff Daddy'
 'Robert Johnson' 'Héroes del Silencio' 'Panic At The Disco']

Прослушанные песни (45 ед.):
['The Cove' 'Jewels And Gold' 'Warning' '83' 'Neon' 'The Middle'
 'All That We Perceive' 'The Christmas Song  (LP Version)'
 'Our Swords (Soundtrack Version)' 'Love Song For No One' 'Are You In?'
 'Generator' 'Come Back To Bed' "He Doesn't Know Why" 'Trani'
 "Bigger Isn't Better" 'Sun Giant' 'City Love' 'Right Back' 

In [15]:
# Выводим список рекомендуемых песен на основе общего рейтинга популярности
# среди Топ-200 самых востребованных песен:
rec.by_popularity(200)

           artist_name                                              title
84      The Black Keys                                         Tighten Up
45          Todd Barry                             Sugar Ray (LP Version)
66               B.o.B  Nothin' On You [feat. Bruno Mars] (Album Version)
167  Eminem / Dina Rae                                           Superman
139   Philippe Rochard                                          Crumpshit
65                Muse                                           Uprising
61            Bon Jovi                                 Livin' On A Prayer
181       Foo Fighters                                      The Pretender
20        Bill Withers                             Make Love To Your Mind
16             Cartola                                           Tive Sim


In [16]:
# Выводим список рекомендуемых песен на основе истории пользователя
# (песни исполнителей, которых пользователь уже слушал):
rec.by_singers()

               artist_name                              title
6588          Foo Fighters                   For All The Cows
284           Jack Johnson                          Breakdown
9076            Kanye West                   Wake Up Mr. West
6545    Panic At The Disco  She Had The World (Album Version)
5748  Thievery Corporation                  A Gentle Dissolve
93          Band Of Horses        The Funeral (Album Version)
181           Foo Fighters                      The Pretender
6009   Angus & Julia Stone                   A Book Like This
2480         Kings Of Leon                               Milk
8516               Incubus                            Vitamin


In [17]:
# Выводим список рекомендуемых песен на основе истории пользователя
# (песни из релизов, композиции из которых пользователь уже слушал):
rec.by_release()

                 artist_name                title
1908201             Harmonia        Sehr kosmisch
230528              Harmonia        Sehr kosmisch
1669245             Harmonia        Sehr kosmisch
955731          Jack Johnson      Do You Remember
1407742             Harmonia        Sehr kosmisch
428263              Harmonia        Sehr kosmisch
908424           Fleet Foxes  Heard Them Stirring
1151156  Angus & Julia Stone                Bella
283336          Jack Johnson            Breakdown
855021             Lady GaGa              Monster


In [18]:
# Выводим список рекомендуемых песен на основе истории пользователя
# (песни, которые слушали пользователи с похожими предпочтениями):
rec.by_similar_users()

                                     artist_name  \
1477840  Aruna Abrams/Ken Hauptman/Jeannie Lurie   
1656332                                 Paramore   
348609                                Lily Allen   
1481501                                  Garbage   
790398                          Five Iron Frenzy   
1481466                            Michael Bublé   
790579                                  Gorillaz   
75264                                   Amos Lee   
790637                          Enrique Iglesias   
1477917                                 Amos Lee   

                                      title  
1477840               I Got Nerve (Karaoke)  
1656332           Emergency (Album Version)  
348609                                  LDN  
1481501                                Milk  
790398                               Canada  
1481466  Spider-Man Theme [Junkie XL Remix]  
790579        Feel Good Inc (Album Version)  
75264                                   Kid  
790637       

In [19]:
# Выводим список рекомендуемых песен, вышедших в период, 
# совпадающий с историей прослушиваний пользователя:
rec.by_period()

                                          artist_name  \
1821986                                   Young Jeezy   
132736                                     Digitalism   
1802309                       Beyoncé feat. Slim Thug   
11098    Destiny's Child featuring T.I. and Lil Wayne   
1804415                                  Eddie Vedder   
500145                            Stone Temple Pilots   
696922                                  Kings Of Leon   
425368                                 Bat For Lashes   
1115579                                  Jack Johnson   
1149388                             Asking Alexandria   

                                                     title  
1821986                                    Streets On Lock  
132736                                           Home Zone  
1802309                                        Check On It  
11098                                              Soldier  
1804415                                    End Of The Road  
500145

In [20]:
# Передаем сведения об ID другого пользователя:
rec.create_user(users[550])

Текущий пользователь: 4eeb2b3849621d81e72010cf6b01147d86220ac7

Прослушанные исполнители (9):
['Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner'
 'J. Karjalainen & Mustat Lasit' 'J. Holiday' 'Todd Barry' 'The Gerbils'
 'Blondie' 'Wiz Khalifa' 'Massive Attack' 'Collie Buddz featuring Roache']

Прослушанные песни (9 ед.):
['Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile)'
 'Sinisten tähtien alla' 'Bed' 'Sugar Ray (LP Version)' '(iii)'
 "Atomic '98 (Xenomania Mix)" 'Say Yeah [Radio Edit]'
 'Paradise Circus (Gui Boratto Remix)' 'Sensimillia']

Временной интервал: с 1998.0 по 2010.0

Похожие пользователи (7948 чел.):
0       0012bf75d43a724f62dc746d9e85ae0088a3a1d6
1       0021d9a4628624f6d70237f9c200ab82e766bf26
2       0028292aa536122c1f86fd48a39bd83fe582d27f
3       002ddb57d13c2a92fa053498c98c1fcb042acdd2
4       0031572620fa7f18487d3ea22935eb28410ecc4c
                          ...                   
7943    ffe2f99d7f8b034a21fafdc2bb859d3c151

In [21]:
# Выводим список рекомендуемых песен на основе общего рейтинга популярности
# среди Топ-100 самых востребованных песен:
rec.by_popularity(100)

                       artist_name  \
94  Kid Cudi / Kanye West / Common   
42                        Coldplay   
50          Florence + The Machine   
9                        Sam Cooke   
14              Charttraxx Karaoke   
79                     The Killers   
82                            Muse   
91                      Katy Perry   
23                      The Crests   
38                        Frumpies   

                                                title  
94                                       Make Her Say  
42                                             Yellow  
50                                        Cosmic Love  
9                                    Ain't Misbehavin  
14                                          Fireflies  
79                                When You Were Young  
82  Supermassive Black Hole (Twilight Soundtrack V...  
91                                    I Kissed A Girl  
23                                         16 Candles  
38                   

In [22]:
# Выводим список рекомендуемых песен на основе истории пользователя
# (песни исполнителей, которых пользователь уже слушал):
rec.by_singers()

         artist_name                                  title
9072  Massive Attack                             Group Four
1441      J. Holiday                                    Bed
2001     Wiz Khalifa                  Say Yeah [Radio Edit]
3311      J. Holiday                              Suffocate
208      The Gerbils                                  (iii)
45        Todd Barry                 Sugar Ray (LP Version)
208      The Gerbils                                  (iii)
9356  Massive Attack  Girl I Love You (She is Danger Remix)
8716         Blondie                  Fade Away And Radiate
9356  Massive Attack  Girl I Love You (She is Danger Remix)


In [23]:
# Выводим список рекомендуемых песен на основе истории пользователя
# (песни из релизов, композиции из которых пользователь уже слушал):
rec.by_release()

                                               artist_name  \
1273864                                     Massive Attack   
805090   Barry Tuckwell/Academy of St Martin-in-the-Fie...   
1812550                                         Todd Barry   
202574                                      Massive Attack   
1676071  Barry Tuckwell/Academy of St Martin-in-the-Fie...   
872219   Barry Tuckwell/Academy of St Martin-in-the-Fie...   
1955584  Barry Tuckwell/Academy of St Martin-in-the-Fie...   
275916                                      Massive Attack   
1577026  Barry Tuckwell/Academy of St Martin-in-the-Fie...   
586023   Barry Tuckwell/Academy of St Martin-in-the-Fie...   

                                                     title  
1273864                                        Rush Minute  
805090   Horn Concerto No. 4 in E flat K495: II. Romanc...  
1812550                             Sugar Ray (LP Version)  
202574                                              Psyche  
1676071  Hor

In [24]:
# Выводим список рекомендуемых песен на основе истории пользователя
# (песни, которые слушали пользователи с похожими предпочтениями):
rec.by_similar_users()

                                               artist_name  \
1561912                                     Olivier Darock   
1411289                                 DAVE MATTHEWS BAND   
443503                                              Atreyu   
114452                                       Amy Winehouse   
262389                                                Nile   
138508                       J. Karjalainen & Mustat Lasit   
222187                                          John Mayer   
1320461  Fort Minor [Featuring Holly Brook And Jonah Ma...   
1261708                                      Steve Goodman   
10163                                              La Roux   

                                                     title  
1561912                                           Miss You  
1411289                           All Along The Watchtower  
443503   Five Vicodin Chased With A Shot Of Clarity (Al...  
114452                                       Wake Up Alone  
262389      

In [25]:
# Выводим список рекомендуемых песен, вышедших в период, 
# совпадающий с историей прослушиваний пользователя:
rec.by_period()

                           artist_name                            title
1337084                      Sean Paul      Temperature (Album Version)
367813                   Drowning Pool      Reason I'm Alive (Explicit)
1478587          Red Hot Chili Peppers  Californication (Album Version)
1879291                       DJ Dizzy                       Sexy Bitch
728419                       Radiohead        Jigsaw Falling Into Place
1419381              Killswitch Engage         My Curse (Album Version)
1999271                             U2                          Walk On
1057818  Under The Influence Of Giants                      Mama's Room
706908         The Cinematic Orchestra                  To Build A Home
646739                    Rise Against             The Good Left Undone
